In [41]:

import pandas as pd
import json
import csv
from collections import defaultdict as ddict

In [42]:
df = pd.read_csv("../raw/retrosheet/teams.csv")

In [43]:
def print_fd(deps, key, dep):
    print(key, dep, sep="\t")
    for k, v in deps.items():
        if len(v) > 1:
            print(f"{k}:=\t{v}")

In [44]:
def fd(pdf, key, dep):
    dets = ddict(set)
    pdf.apply(lambda x: dets[x[key]].add(x[dep]), axis=1)
    print_fd(dets, key, dep)
    return dets

In [45]:
deps = fd(df, "teamID", "teamIDretro")
# deps = fd(df, "teamIDretro", "teamID")

teamID	teamIDretro
WAS:=	{'WSN', 'WAS'}
LAA:=	{'ANA', 'LAA'}


In [46]:
def replace_fd(pdf, index, key, rmap):
    if pdf.loc[index, key] in rmap:
        pdf.loc[index, key] = rmap[pdf.loc[index, key]]

In [47]:
rretroIDMap = {
    "WSN": "WAS",
    "ANA": "LAA",
}

In [48]:
# change all ANA in teamIDretro to LAA
# change all WSN in teamIDretro to WAS

for index in df.index:
    replace_fd(df, index, "teamIDretro", rretroIDMap)

In [49]:
deps = fd(df, "teamID", "teamIDretro")

teamID	teamIDretro


In [50]:
# drop lgID, franchID, divID, Rank, G, Ghome, DivWin WCWin, LgWin, WSWin column
df.drop(columns=["lgID", "franchID", "divID", "Rank", "G", "Ghome", "DivWin", "WCWin", "LgWin", "WSWin"], inplace=True)

In [60]:
df_idMap = df[["teamID", "teamIDretro"]].drop_duplicates()
df_idMap.to_csv("../raw/retrosheet/teamIDMap.csv", index=False)

df_retIdMap = df_idMap[["teamIDretro"]].drop_duplicates()
df_retIdMap["ID"] = range(len(df_retIdMap))
df_retIdMap.to_csv("../raw/retrosheet/teamIDretroMap.csv", index=False)

In [61]:
df.keys()

Index(['yearID', 'teamID', 'W', 'L', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB',
       'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV',
       'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP', 'name', 'park',
       'attendance', 'BPF', 'PPF', 'teamIDBR', 'teamIDlahman45',
       'teamIDretro'],
      dtype='object')

In [62]:
to_drop = ['teamID', 'W', 'L', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB',
       'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV',
       'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'BPF', 'PPF', 'teamIDBR', 'teamIDlahman45']
df.drop(columns=to_drop, inplace=True)

In [63]:
df_teamnames = df[["yearID", "teamIDretro", "name"]].drop_duplicates()
df_teamnames.to_csv("../raw/retrosheet/teamnames.csv", index=False)

In [64]:
rMapTeamIDRetro = {
    row["teamIDretro"]: row["ID"] for _, row in df_retIdMap.iterrows()
}

In [65]:
for index in df.index:
    replace_fd(df, index, "teamIDretro", rMapTeamIDRetro)

for index in df_idMap.index:
    replace_fd(df_idMap, index, "teamIDretro", rMapTeamIDRetro)

for index in df_teamnames.index:
    replace_fd(df_teamnames, index, "teamIDretro", rMapTeamIDRetro)

In [66]:
df_teamnames.rename(columns={"teamIDretro": "ID", "yearID": "year"}, inplace=True)
df_idMap.rename(columns={"teamIDretro": "ID"}, inplace=True)

In [67]:
df_teamnames.to_csv("../processed/data/teamnames.csv", index=False)
df_idMap.to_csv("../processed/data/teamIDMap.csv", index=False)

In [68]:
df.rename(columns={"teamIDretro": "ID", "yearID": "year"}, inplace=True)

In [70]:
_ = fd(df, "park", "ID")

park	ID
South End Grounds I:=	{0, 26}
Union Grounds (Brooklyn):=	{4, 29, 31}
Jefferson Street Grounds:=	{16, 32, 5, 46}
Haymakers' Grounds:=	{40, 7}
Olympics Grounds:=	{8, 17, 25}
Newington Park:=	{9, 43, 14}
Union Grounds:=	{10, 11}
23rd Street Grounds:=	{18, 27}
Hartford Ball Club Grounds:=	{19, 29}
Sportsman's Park I:=	{48, 33, 100, 70}
Eclipse Park II:=	{99, 35}
Bank Street Grounds:=	{28, 44}
Recreation Park:=	{42, 75, 52, 47}
Eclipse Park I:=	{99, 45}
Oriole Park:=	{80, 43, 102}
nan:=	{53, 54, 56, 57, 58, 60, 61, 62, 64, 65, 67, 69, 79, 83, 84, 85, 87, 88, 89, 90, 112, 113, 114, 116, 117, 118, 119, 120}
League Park I in Cincinnati:=	{44, 86}
Athletic Park:=	{68, 46}
Exposition Park:=	{75, 76}
Union Park:=	{80, 98}
League Park I:=	{105, 77}
Boundary Field:=	{97, 101}
Shibe Park:=	{108, 52}
Sportsman's Park IV:=	{100, 110}
Polo Grounds IV:=	{50, 130, 111}
Wrigley Field:=	{27, 115}
County Stadium:=	{121, 138}
Connie Mack Stadium:=	{108, 52}
Griffith Stadium II:=	{128, 109}
Dodger Sta

In [71]:
df.drop(columns=["park"], inplace=True)

In [73]:
df_team_attendance = df[["year", "ID", "attendance"]].drop_duplicates()
df_team_attendance.to_csv("../processed/data/team_attendance.csv", index=False)

In [74]:

df.drop(columns=["name", "year", "attendance"], inplace=True)

In [76]:
df.drop_duplicates(inplace=True)
df.to_csv("../processed/data/team.csv", index=False)